In [1]:
import numpy as np
import pandas as pd

import seaborn as sns

In [3]:
dataset = sns.load_dataset('tips')

In [4]:
dataset.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [6]:
dataset['sex'].unique()

['Female', 'Male']
Categories (2, object): ['Male', 'Female']

In [ ]:
## handling Missing values (no null values)
## handling categorical features

In [8]:
dataset.shape

(244, 7)

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
encoder = LabelEncoder()

In [12]:
dataset['time'] = encoder.fit_transform(dataset['time'])

In [13]:
dataset.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [14]:
dataset.time.value_counts()

0    176
1     68
Name: time, dtype: int64

In [16]:
## segregate the independent and dependent feature

X = dataset.drop('time',axis=1)
y = dataset['time']

In [17]:
X

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [18]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y, test_size=0.2,random_state=42)

In [19]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [20]:
dataset.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [23]:
category_column = ['sex','smoker','day']
numeric_column =  [ "total_bill" , 'tip' , 'size']

In [26]:
numeric_pipeline= Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

cat_pipeline= Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('Onehotencoder',OneHotEncoder())
    ]
)

In [27]:
preprocessor = ColumnTransformer(
    [
        ('num_pipeline',numeric_pipeline,numeric_column),
        ('cat_pipeline',cat_pipeline,category_column)
    ]
)

In [28]:
X_train = preprocessor.fit_transform(X_train)

In [29]:
X_test = preprocessor.transform(X_test)

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [31]:
models = {

    "RandomForest":RandomForestClassifier(),
    "DecisionTree":DecisionTreeClassifier(),
    'SVC':SVC()
}

In [68]:
def evaluate_model(X_train,X_test,y_train,y_test,models):

    report= {}
    for i in range(len(models)):
        model = list(models.values())[i]
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        score = accuracy_score(y_test,y_pred)
        score = round(score*100,2)
        report[list(models.keys())[i]] = score

    best_score = max(sorted(report.values()))
    best_model = list(report.keys())[list(report.values()).index(best_score)]

    print("\n===============================================================")
    print(f"The best model is {best_model} : and its score is {best_score}")
    print("\n===============================================================")

    
    return report

In [69]:
evaluate_model(X_train,X_test,y_train,y_test,models)


The best model is RandomForest : and its score is 95.92



{'RandomForest': 95.92, 'DecisionTree': 93.88, 'SVC': 95.92}

In [67]:
list(d.keys())[list(d.values()).index(best_score)]

'RandomForest'

In [49]:
classifier = RandomForestClassifier()

In [56]:
params = {
    'max_depth':[3,5,8,12],
    "n_estimators": [100,300,600],
    'criterion': ['gini','entropy'],
    'max_features': ['sqrt','log2']
}

In [59]:
from sklearn.model_selection import GridSearchCV
hp_model = GridSearchCV(classifier,param_grid=params,scoring='accuracy',cv=5, verbose=3, n_jobs=-1)

In [60]:
hp_model.fit(X_train,y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 8, 12],
                         'n_estimators': [100, 300, 600]},
             scoring='accuracy', verbose=3)

In [61]:
hp_model.best_params_

{'criterion': 'gini', 'max_depth': 3, 'n_estimators': 100}

In [70]:
classifier = RandomForestClassifier(criterion='gini',max_depth=3,n_estimators=100)

In [71]:
classifier.fit(X_train,y_train)

RandomForestClassifier(max_depth=3)

In [76]:
classifier.score(X_train,y_train)

0.958974358974359

In [73]:
y_pred = classifier.predict(X_test)

In [74]:
score = accuracy_score(y_test,y_pred)
print(f'accuracy_score:{score}')

accuracy_score:1.0
